# Assignment 3 (Concise, without reading data from external source)



$$ \sum_{i=A,...G,j=1,..,4} (prod\_cost_{i,j}+trans\_cost_{i,j})x_{i,j}\\
s.t. \\
Demand: \sum_{i=A,...,G}x_{ij} \ge Demand_j,  j=1,2,3,4\\
Capacity: \sum_{j=1,...,4}proc\_time_{i,j}x_{i,j}\le i=A,...,G \\
A1: x_{A,1}=0 \\
D1: x_{D,1}=0 \\
$$

## Import PuLP modeler functions

In [6]:
from pulp import *
#import pandas as pd
#import numpy as np

## Fixed Cost

In [15]:
probA=LpProblem("Problem A",LpMinimize)

factory=['A','B','C','D','E','F','G']

yarn = ['1','2','3','4']


# Creates a list of production cost
prod_cost =[ [0,13.00, 10.65, 9.60],\
            [ 17.40,14.10,11.20,9.45],\
            [ 17.40,14.22,11.00,9.50],\
            [0,14.30,11.25,9.60],\
            [ 17.50, 13.80,11.40,9.60],\
            [ 18.25, 13.90,11.40, 8.90],\
            [ 19.75,13.90,10.75,9.40 ]]

prod_cost= makeDict([factory,yarn],prod_cost)

# Creates a list of transportation cost
trans_cost = [
    [ 0.30,0.30,0.45, 0.45],\
    [ 0.40,0.40,0.60,0.60],\
    [ 0.80,0.80, 1.20,1.20],\
    [ 0.70,0.70,1.05,1.05],\
    [ 0.70, 0.70, 1.05, 1.05],\
    [ 0,0,0,0],\
    [ 0.50, 0.50, 0.75, 0.75]]

trans_cost= makeDict([factory,yarn],trans_cost)

proc_time=[
    [0, 0.400,0.375,0.250],\
    [0.700,0.500,0.350,0.250],\
    [0.675,0.450,0.400,0.250],\
    [0,0.450,0.350,0.200],\
    [0.650,0.450,0.400,0.250],\
    [0.625,0.500,0.425,0.425],\
    [0.700,0.450,0.350,0.400],\
]
proc_time= makeDict([factory,yarn],proc_time)
capacity = {
    "A":2500,
"B":3000,
"C":2500,
"D":2600,
"E":2500,
"F":38000,
"G":2500}
demand={
    "1":25000,
    "2":26000,
    "3":28000,
    "4":28000
}

fixed_cost ={
    "A":100000,
"B":100000,
"C":100000,
"D":100000,
"E":100000,
"F":0,
"G":100000
}

production_vars = LpVariable.dicts("x", (factory,yarn),lowBound=0, cat='Continuous')
source_vars = LpVariable.dicts("y", factory,lowBound=0, cat='Binary')

### Fixed Cost: Add objective function and constraints
- Use lpSum() amd += operator to construct objective function and constraintsumn based modelling: relates to the variable’s existence in the objective function and constraints

In [16]:
#objective function
probA+=lpSum([fixed_cost[i]*source_vars[i] for i in factory])+lpSum([production_vars[i][j]*(prod_cost[i][j]+trans_cost[i][j]) for i in factory for j in yarn]),"total cost"
# demand constraint
for j in yarn:
    probA += lpSum([production_vars[i][j] for i in factory ]) >= demand[j],"demand_%s"%j

# capacity constraint    
for i in factory:
    probA += lpSum([proc_time[i][j]*production_vars[i][j] for j in yarn]) <= capacity[i]*source_vars[i],"capacity_%s"%i

# x_1A and x_1D constraint
probA += production_vars["A"]["1"]==0,"xA1=0"
probA += production_vars["D"]["1"]==0,"xD1=0"

### Fixed Cost: Run solver

- use name.solve(solver=None), where name is the LP problem variable defined by LpProblem
- Solve the given Lp problem. 
- This function changes the problem to make it suitable for solving then calls the solver.actualSolve method to find the solution. 
- solver – Optional: the specific solver to be used, defaults to the default solver.

In [17]:
probA.writeLP("FilatoiMILP.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


### Fixed Cost: Print the optiomal solution

In [18]:
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probA.objective))
original_cost=value(probA.objective)

x_A_1 = 0.0 	Reduced Cost = 0.0
x_A_2 = 6250.0 	Reduced Cost = 0.0
x_A_3 = 0.0 	Reduced Cost = 0.55931373
x_A_4 = 0.0 	Reduced Cost = 0.76176471
x_B_1 = 0.0 	Reduced Cost = 0.27647059
x_B_2 = 0.0 	Reduced Cost = 0.75546218
x_B_3 = 8571.4286 	Reduced Cost = 0.0
x_B_4 = 0.0 	Reduced Cost = 0.04047619
x_C_1 = 0.0 	Reduced Cost = 0.0
x_C_2 = 0.0 	Reduced Cost = 0.58797386
x_C_3 = 0.0 	Reduced Cost = 0.45900508
x_C_4 = 0.0 	Reduced Cost = 0.50562818
x_D_1 = 0.0 	Reduced Cost = 0.0
x_D_2 = 0.0 	Reduced Cost = 0.25784314
x_D_3 = 2834.7339 	Reduced Cost = 0.0
x_D_4 = 8039.2157 	Reduced Cost = 0.0
x_E_1 = 0.0 	Reduced Cost = 0.0
x_E_2 = 0.0 	Reduced Cost = 0.17797888
x_E_3 = 0.0 	Reduced Cost = 0.80678733
x_E_4 = 0.0 	Reduced Cost = 0.51674208
x_F_1 = 25000.0 	Reduced Cost = 0.0
x_F_2 = 19750.0 	Reduced Cost = 0.0
x_F_3 = 9450.9804 	Reduced Cost = 0.0
x_F_4 = 19960.784 	Reduced Cost = 0.0
x_G_1 = 0.0 	Reduced Cost = 3.3264706
x_G_2 = 0.0 	Reduced Cost = 0.68641457
x_G_3 = 7142.8571 	Reduced Cos

In [143]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)


Sensitivity Analysis
Constraint		Shadow Price	Slack
demand_1 : x_A_1 + x_B_1 + x_C_1 + x_D_1 + x_E_1 + x_F_1 + x_G_1 >= 25000 	 19.573529 		 -0.0
demand_2 : x_A_2 + x_B_2 + x_C_2 + x_D_2 + x_E_2 + x_F_2 + x_G_2 >= 26000 	 14.958824 		 -0.0
demand_3 : x_A_3 + x_B_3 + x_C_3 + x_D_3 + x_E_3 + x_F_3 + x_G_3 >= 28000 	 12.3 		 -0.0
demand_4 : x_A_4 + x_B_4 + x_C_4 + x_D_4 + x_E_4 + x_F_4 + x_G_4 >= 28000 	 9.8 		 -0.0
capacity_A : 0.4*x_A_2 + 0.375*x_A_3 + 0.25*x_A_4 <= 2500.0 	 -4.1470588 		 -0.0
capacity_B : 0.7*x_B_1 + 0.5*x_B_2 + 0.35*x_B_3 + 0.25*x_B_4 <= 3000.0 	 -2.5336134 		 -0.0
capacity_C : 0.675*x_C_1 + 0.45*x_C_2 + 0.4*x_C_3 + 0.25*x_C_4 <= 2500.0 	 -2.0348584 		 -0.0
capacity_D : 0.45*x_D_2 + 0.35*x_D_3 + 0.2*x_D_4 <= 2600.0 	 0.0 		 1885.9560900000001
capacity_E : 0.65*x_E_1 + 0.45*x_E_2 + 0.4*x_E_3 + 0.25*x_E_4 <= 2500.0 	 -2.1131222 		 -0.0
capacity_F : 0.625*x_F_1 + 0.5*x_F_2 + 0.425*x_F_3 + 0.425*x_F_4 <= 38000.0 	 -2.1176471 		 -0.0
capacity_G : 0.7*x_G_1 + 0.45*x_G_2 + 

## Minimum Requirement 5000KG

In [19]:
probB=LpProblem("Problem B",LpMinimize)

factory=['A','B','C','D','E','F','G']

yarn = ['1','2','3','4']


# Creates a list of production cost
prod_cost =[ [0,13.00, 10.65, 9.60],\
            [ 17.40,14.10,11.20,9.45],\
            [ 17.40,14.22,11.00,9.50],\
            [0,14.30,11.25,9.60],\
            [ 17.50, 13.80,11.40,9.60],\
            [ 18.25, 13.90,11.40, 8.90],\
            [ 19.75,13.90,10.75,9.40 ]]

prod_cost= makeDict([factory,yarn],prod_cost)

# Creates a list of transportation cost
trans_cost = [
    [ 0.30,0.30,0.45, 0.45],\
    [ 0.40,0.40,0.60,0.60],\
    [ 0.80,0.80, 1.20,1.20],\
    [ 0.70,0.70,1.05,1.05],\
    [ 0.70, 0.70, 1.05, 1.05],\
    [ 0,0,0,0],\
    [ 0.50, 0.50, 0.75, 0.75]]

trans_cost= makeDict([factory,yarn],trans_cost)

proc_time=[
    [0, 0.400,0.375,0.250],\
    [0.700,0.500,0.350,0.250],\
    [0.675,0.450,0.400,0.250],\
    [0,0.450,0.350,0.200],\
    [0.650,0.450,0.400,0.250],\
    [0.625,0.500,0.425,0.425],\
    [0.700,0.450,0.350,0.400],\
]
proc_time= makeDict([factory,yarn],proc_time)
capacity = {
    "A":2500,
"B":3000,
"C":2500,
"D":2600,
"E":2500,
"F":38000,
"G":2500}
demand={
    "1":25000,
    "2":26000,
    "3":28000,
    "4":28000
}

min_req={
    "A":5000,
"B":5000,
"C":5000,
"D":5000,
"E":5000,
"F":5000,
"G":5000}
production_vars = LpVariable.dicts("x", (factory,yarn),lowBound=0, cat='Continuous')
source_vars = LpVariable.dicts("y", factory,lowBound=0, cat='Binary')

In [20]:
#objective function
probB+=lpSum([production_vars[i][j]*(prod_cost[i][j]+trans_cost[i][j]) for i in factory for j in yarn]),"total cost"
# demand constraint
for j in yarn:
    probB += lpSum([production_vars[i][j] for i in factory ]) >= demand[j],"demand_%s"%j

# capacity constraint    
for i in factory:
    probB += lpSum([proc_time[i][j]*production_vars[i][j] for j in yarn]) <= capacity[i]*source_vars[i],"capacity_%s"%i
    probB += lpSum([production_vars[i][j] for j in yarn ]) >= min_req[i]*source_vars[i],"min_req_%s"%i
# x_1A and x_1D constraint
probB += production_vars["A"]["1"]==0,"xA1=0"
probB += production_vars["D"]["1"]==0,"xD1=0"

In [21]:
probB.writeLP("FilatoiMILP2.lp")
probB.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


In [22]:
for v in probB.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probB.objective))

x_A_1 = 0.0 	Reduced Cost = 0.0
x_A_2 = 6250.0 	Reduced Cost = 0.0
x_A_3 = 0.0 	Reduced Cost = 0.50555556
x_A_4 = 0.0 	Reduced Cost = 0.9
x_B_1 = 3324.6965 	Reduced Cost = 0.0
x_B_2 = 0.0 	Reduced Cost = 0.62380952
x_B_3 = 1922.0355 	Reduced Cost = 0.0
x_B_4 = 0.0 	Reduced Cost = 0.18968254
x_C_1 = 2941.1765 	Reduced Cost = 0.0
x_C_2 = 0.0 	Reduced Cost = 0.29679739
x_C_3 = 0.0 	Reduced Cost = 0.045098039
x_C_4 = 2058.8235 	Reduced Cost = 0.0
x_D_1 = 0.0 	Reduced Cost = 0.0
x_D_2 = 0.0 	Reduced Cost = 0.20079365
x_D_3 = 0.0 	Reduced Cost = 0.0
x_D_4 = 0.0 	Reduced Cost = 0.22380952
x_E_1 = 1250.0 	Reduced Cost = 0.0
x_E_2 = 3750.0 	Reduced Cost = 0.0
x_E_3 = 0.0 	Reduced Cost = 0.53055556
x_E_4 = 0.0 	Reduced Cost = 0.22222222
x_F_1 = 17484.127 	Reduced Cost = 0.0
x_F_2 = 16000.0 	Reduced Cost = 0.0
x_F_3 = 18935.107 	Reduced Cost = 0.0
x_F_4 = 25941.176 	Reduced Cost = 0.0
x_G_1 = 0.0 	Reduced Cost = 3.05
x_G_2 = 0.0 	Reduced Cost = 0.62936508
x_G_3 = 7142.8571 	Reduced Cost = 0.0
x_G

In [ ]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probB.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)